![Class Relationship](../figs/classrelationship.png)

In [ ]:
# 我们的 AdvancedMutationFuzzer 类是 MutationFuzzer 类的高级和参数化版本，该类来自基于突变的模糊测试章节。它也继承自 Fuzzer 类。目前，我们只需要知道函数 fuzz()，它返回生成的输入，以及函数 runs()，它执行 fuzz() 指定次数。对于我们的 AdvancedMutationFuzzer 类，我们重写了函数 fuzz()。
from fuzzingbook.Fuzzer import Fuzzer
from typing import List, Set, Any, Tuple, Dict, Union
from fuzzingbook.GreyboxFuzzer import Seed, Mutator, PowerSchedule
import random
from fuzzingbook.Coverage import population_coverage

class AdvancedMutationFuzzer(Fuzzer):
    """Base class for mutation-based fuzzing."""

    def __init__(self, seeds: List[str],
                 mutator: Mutator,
                 schedule: PowerSchedule) -> None:
        """Constructor.
        `seeds` - a list of (input) strings to mutate.
        `mutator` - the mutator to apply.
        `schedule` - the power schedule to apply.
        """
        self.seeds = seeds
        self.mutator = mutator
        self.schedule = schedule
        self.inputs: List[str] = []
        self.reset()

    def reset(self) -> None:
        """Reset the initial population and seed index"""
        self.population = list(map(lambda x: Seed(x), self.seeds))
        self.seed_index = 0

    def create_candidate(self) -> str:
        """Returns an input generated by fuzzing a seed in the population"""
        seed = self.schedule.choose(self.population)

        # Stacking: Apply multiple mutations to generate the candidate
        candidate = seed.data
        trials = min(len(candidate), 1 << random.randint(1, 5))
        for i in range(trials):
            candidate = self.mutator.mutate(candidate)
        return candidate

    def fuzz(self) -> str:
        """Returns first each seed once and then generates new inputs"""
        if self.seed_index < len(self.seeds):
            # Still seeding
            self.inp = self.seeds[self.seed_index]
            self.seed_index += 1
        else:
            # Mutating
            self.inp = self.create_candidate()

        self.inputs.append(self.inp)
        return self.inp
    


seed_input = "ELonMusk"
mutation_fuzzer = AdvancedMutationFuzzer([seed_input], Mutator(), PowerSchedule())
print(mutation_fuzzer.fuzz())
print(mutation_fuzzer.fuzz())
print(mutation_fuzzer.fuzz())

ELonMusk
ELnMu('}s{
L0o|Eusk


In [6]:
def crashme(s: str) -> None:
    if len(s) > 0 and s[0] == 'b':
        if len(s) > 1 and s[1] == 'a':
            if len(s) > 2 and s[2] == 'd':
                if len(s) > 3 and s[3] == '!':
                    raise Exception()

让我们看看基于突变的黑盒模糊测试器在具有n = 30k个输入的测试活动中覆盖了多少个语句。

模糊测试器函数`runs(crashme_runner, trials=n)`生成`n`个输入，并通过`crashme_runner`在`crashme`函数上执行它们。如前所述，`crashme_runner`还收集覆盖率信息。

In [ ]:
import time
from fuzzingbook.MutationFuzzer import FunctionCoverageRunner
n = 30000
# 默认的PowerSchedule,即把每个种子的能量（权重）都为1
blackbox_fuzzer = AdvancedMutationFuzzer([seed_input], Mutator(), PowerSchedule())

start = time.time()
crashme_runner = FunctionCoverageRunner(crashme)
blackbox_fuzzer.runs(crashme_runner, trials=n)
# or
# blackbox_fuzzer.runs(FunctionCoverageRunner(crashme), trials=n)
end = time.time()

"It took the blackbox mutation-based fuzzer %0.2f seconds to generate and execute %d inputs." % (end - start, n)

'It took the blackbox mutation-based fuzzer 4.29 seconds to generate and execute 30000 inputs.'

In [9]:
_, blackbox_coverage = population_coverage(blackbox_fuzzer.inputs, crashme)
bb_max_coverage = max(blackbox_coverage)

"The blackbox mutation-based fuzzer achieved a maximum coverage of %d statements." % bb_max_coverage


'The blackbox mutation-based fuzzer achieved a maximum coverage of 2 statements.'